In [11]:
import numpy as np
import keras

In [12]:
from keras.models import Sequential
from keras.layers import Dense

In [13]:
def load_file(filename):
    
    #return np.array(np.load(open(filename, 'r')), dtype='float32')
    
    csv = np.array(np.genfromtxt(filename, delimiter=","), dtype='float32')
    return csv

In [14]:
# split data before scaling

def split_train_val(percent, data):
    num = int(percent/100*len(data))
    train, test = data[:num,:], data[num:,:]
    return train, test

In [15]:
def combine(one,two):
    return np.concatenate((one,two), axis=0)

In [16]:
def create_binary_labels(num):
    return np.array([0]*num + [1]*num)

In [17]:
# find only for training data

def colmin(data):
    mins = []
    
    for i in range(len(data[0])):
        mins.append(min(data[:,i]))
        
    return mins

In [18]:
# find only for training data

def colmax(data):
    maxs = []
    
    for i in range(len(data[0])):
        maxs.append(max(data[:,i]))
    
    return maxs

In [19]:
# use values from training data to scale everything, split first then scale

def applyMinMax(data, mins, maxs):
    
    for i in range(len(data)):
        for t in range(len(data[0])):
            data[i][t] = (data[i][t]-mins[t])/(maxs[t]-mins[t])
            
    return data

In [34]:
leon = load_file('leon.csv')
chino = load_file('chino.csv')
leon[0]

array([  1.00000000e+00,   3.40000000e+01,   7.20300000e+07,
         6.39990000e+07], dtype=float32)

In [35]:
leon[:,2] = leon[:,2]/leon[:,3]
leon = leon[:,:3]
chino[:,2] = chino[:,2]/chino[:,3]
chino = chino[:,:3]

In [36]:
leon[0]

array([  1.        ,  34.        ,   1.12548637], dtype=float32)

In [37]:
leon_train, leon_test = split_train_val(80,leon)
chino_train, chino_test = split_train_val(80,chino)

In [38]:
chino_train.shape

(120326, 3)

In [39]:
train = combine(leon_train, chino_train)
test = combine(leon_test, chino_test)

In [40]:
train.shape

(240652, 3)

In [41]:
y_train = create_binary_labels(leon_train.shape[0])
y_test = create_binary_labels(leon_test.shape[0])

In [42]:
y_train[120326]

1

In [43]:
minimum = colmin(train)

In [44]:
minimum

[0.0, 0.0, 9.3754979e-06]

In [45]:
maximum = colmax(train)

In [46]:
maximum

[126.0, 126.0, 54674.336]

In [47]:
normalized_train = applyMinMax(train, minimum, maximum)

In [48]:
normalized_test = applyMinMax(test, minimum, maximum)

In [4]:
model = Sequential()

model.add(Dense(256, input_shape=(3,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [5]:
model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

In [ ]:
model.fit(normalized_train, y_train, verbose=1, batch_size=128, epochs=20, validation_data=(normalized_test, y_test))

In [ ]:
model.save('keyboard.h5')